In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from IPython.display import display
pd.set_option('max_rows', 5)

# スケーリング (scaling)
---
特徴量の尺度を揃えること。多くの機械学習アルゴリズム (特に kNN などの特徴空間内の距離に基づくもの) は、特徴量の尺度が揃っていないとうまく動かない。

<table class="border">
    <tr>
        <th class="text-center border-bottom">正則化</th>
        <td class="text-left">係数の値の合計に基づくため、スケールの小さい (係数の大きい) 変数ほど抑制されやすい</td>
    </tr>
    <tr>
        <th class="text-center border-bottom">kNN</th>
        <td class="text-left">特徴空間内の距離に基づくため、スケールの大きい変数に距離が影響されやすい</td>
    </tr>
    <tr>
        <th class="text-center border-bottom">PCA</th>
        <td class="text-left">変数の共分散に基づくため、スケールの大きい変数ほど上位の成分に多く含まれやすい</td>
    </tr>
    <tr>
        <th class="text-center border-bottom">SGD</th>
        <td class="text-left">パラメーター空間内の勾配に基づくため、スケールの大きい (パラメーターの勾配が急な) 変数方向に行き来を繰り返して収束が遅くなりやすい</td>
    </tr>
</table>

トレーニングデータを変換するのに使用したのと同じパラメーターで、未知のデータも変換しないといけない。

## 正規化 (normalization)
---
対象となる特徴の最小値・最大値を0・1になるように変換すること。その特徴のとる値の範囲が一定範囲内に限定される場合に有効。

$
{\displaystyle x^{( i)} =\frac
    {x^{( i)} -x_{min}}
    {x_{max} -x_{min}}
}
$

練習問題

---
scikit-learn を用いずに`sample_n11n`データセットを正規化する。その後、正規化されたデータと最大値・最小値のパラメーターを用いて、元の値に戻す。

In [2]:
np.random.seed(1234)
sample_n11n = pd.Series(np.random.uniform(low=-10, high=10, size=10),
                        name='sample_n11n')
print('sample_n11n')
display(sample_n11n)

sample_n11n


0   -6.169611
1    2.442175
       ...   
8    9.162787
9    7.518653
Name: sample_n11n, Length: 10, dtype: float64

解答例

---

In [3]:
sample_min = sample_n11n.min()
sample_max = sample_n11n.max()

norm = (sample_n11n - sample_min) / (sample_max - sample_min)
print(norm)

0    0.000000
1    0.561673
       ...   
8    1.000000
9    0.892767
Name: sample_n11n, Length: 10, dtype: float64


In [4]:
original = norm * (sample_max - sample_min) + sample_min
print(original)

0   -6.169611
1    2.442175
       ...   
8    9.162787
9    7.518653
Name: sample_n11n, Length: 10, dtype: float64


---

### Pythonでの実行方法
---
`sklearn.preprocessing.MinMaxScaler`を使用する。

In [5]:
help(MinMaxScaler)

Help on class MinMaxScaler in module sklearn.preprocessing._data:

class MinMaxScaler(sklearn.base.TransformerMixin, sklearn.base.BaseEstimator)
 |  MinMaxScaler(feature_range=(0, 1), *, copy=True)
 |  
 |  Transform features by scaling each feature to a given range.
 |  
 |  This estimator scales and translates each feature individually such
 |  that it is in the given range on the training set, e.g. between
 |  zero and one.
 |  
 |  The transformation is given by::
 |  
 |      X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
 |      X_scaled = X_std * (max - min) + min
 |  
 |  where min, max = feature_range.
 |  
 |  This transformation is often used as an alternative to zero mean,
 |  unit variance scaling.
 |  
 |  Read more in the :ref:`User Guide <preprocessing_scaler>`.
 |  
 |  Parameters
 |  ----------
 |  feature_range : tuple (min, max), default=(0, 1)
 |      Desired range of transformed data.
 |  
 |  copy : bool, default=True
 |      Set to False to perfor

In [6]:
n11n_reshape = sample_n11n.values.reshape((-1, 1))
mm_scaler = MinMaxScaler().fit(n11n_reshape)
# 結果の確認
mm_scaler.transform(n11n_reshape)

array([[0.        ],
       [0.5616725 ],
       [0.32116084],
       [0.77462003],
       [0.76759859],
       [0.10575404],
       [0.11080433],
       [0.79616082],
       [1.        ],
       [0.8927673 ]])

## 標準化 (standardization)
---
対象となる特徴の分布が平均0・標準偏差1の正規分布となるように変換すること。多くの機械学習モデルの前処理として有効。正規化と比べて外れ値から受ける影響が少なくなる。

標準化のことも正規化ということがあるので、文脈からどちらを指しているのか判断する。

$
{\displaystyle x^{( i)} =\frac
    {x^{( i)} -\mu _{x}}
    {\sigma _{x}}
}
$

練習問題

---
scikit-learn を用いずに`sample_s13n`データセットを標準化する。その後、標準化されたデータと平均・標準偏差のパラメータを用いて元の値に戻す。

In [7]:
np.random.seed(1234)
sample_s13n = pd.Series(np.random.normal(loc=5, scale=2, size=10))
print('sample_s13n')
display(sample_s13n)

sample_s13n


0    5.942870
1    2.618049
       ...   
8    5.031393
9    0.514630
Length: 10, dtype: float64

解答例

---

In [8]:
sample_mean = sample_s13n.mean()
sample_std = sample_s13n.std()

std = (sample_s13n - sample_mean) / sample_std
print(std)

0    0.554762
1   -0.944531
       ...   
8    0.143741
9   -1.893045
Length: 10, dtype: float64


In [9]:
original = std * sample_std + sample_mean
print(original)

0    5.942870
1    2.618049
       ...   
8    5.031393
9    0.514630
Length: 10, dtype: float64


---

### Pythonでの実行方法
---
`sklearn.preprocessing.StandardScaler`を使用する。

In [10]:
help(StandardScaler)

Help on class StandardScaler in module sklearn.preprocessing._data:

class StandardScaler(sklearn.base.TransformerMixin, sklearn.base.BaseEstimator)
 |  StandardScaler(*, copy=True, with_mean=True, with_std=True)
 |  
 |  Standardize features by removing the mean and scaling to unit variance
 |  
 |  The standard score of a sample `x` is calculated as:
 |  
 |      z = (x - u) / s
 |  
 |  where `u` is the mean of the training samples or zero if `with_mean=False`,
 |  and `s` is the standard deviation of the training samples or one if
 |  `with_std=False`.
 |  
 |  Centering and scaling happen independently on each feature by computing
 |  the relevant statistics on the samples in the training set. Mean and
 |  standard deviation are then stored to be used on later data using
 |  :meth:`transform`.
 |  
 |  Standardization of a dataset is a common requirement for many
 |  machine learning estimators: they might behave badly if the
 |  individual features do not more or less look like s

In [11]:
s13n_reshape = sample_s13n.values.reshape((-1, 1))
std_scaler = StandardScaler().fit(s13n_reshape)
# 結果の確認
std_scaler.transform(s13n_reshape)

array([[ 0.58477094],
       [-0.99562261],
       [ 1.49861708],
       [-0.16063212],
       [-0.54844283],
       [ 0.97998822],
       [ 0.95377412],
       [-0.46852508],
       [ 0.15151664],
       [-1.99544436]])

## 推薦図書
---
- [Python 機械学習プログラミング 達人データサイエンティストによる理論と実践](https://www.amazon.co.jp/Python-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0-%E9%81%94%E4%BA%BA%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%83%86%E3%82%A3%E3%82%B9%E3%83%88%E3%81%AB%E3%82%88%E3%82%8B%E7%90%86%E8%AB%96%E3%81%A8%E5%AE%9F%E8%B7%B5-impress-gear/dp/4295003379/)